In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from dl.dataset.datamodes.npz import rectifiedrgb, debayeredrgb
from dl.config.label_map_helper import LabelMapHelper, LabelConversion
from collections import defaultdict
import pandas as pd
import os
from pathlib import Path
import json
from tqdm import tqdm
import seaborn

In [2]:
model_output_dirs = [
    Path('/mnt/sandbox1/alex.li/introspection/25079_alleyson'),
    Path('/mnt/sandbox1/alex.li/introspection/25080_alleyson'),
]

In [3]:
out_csvs = defaultdict(lambda:{})
for model_dir in model_output_dirs:
    dsets = os.listdir(model_dir)
    for dset in dsets:
        path = model_dir / dset / 'results.txt'
        if os.path.exists(path):
            print(path)
            with open(path, 'r') as f:
                print(''.join(f.readlines()))
        if os.path.exists(model_dir / dset / 'output.csv'):
            out_csvs[model_dir.name][dset] = pd.read_csv(model_dir / dset / 'output.csv')

/mnt/sandbox1/alex.li/introspection/25079_alleyson/halo_productivity_combined_no_mislocalization_alleyson/results.txt

General metrics. Note TPs and FNs are based on gt_classes ['Non-driveable', 'Trees_Weeds', 'Humans', 'Vehicles']                 and pred_classes ['Non-driveable', 'Trees_Weeds', 'Humans', 'Vehicles']:
true_positive: 0
true_negative: 180576
false_positive: 3387
false_negative: 0
precision_image: 0.0
recall_image: nan
f1_image: 0.0
productivity_image: 0.9815886890298593

Strict metrics on humans:
human_true_positive: 0
human_true_negative: 181305
human_false_positive: 2658
human_false_negative: 0
human_precision_image: 0.0
human_recall_image: nan
human_f1_image: 0.0
human_productivity_image: 0.985551442409615

Strict metrics on vehicles:
vehicle_true_positive: 0
vehicle_true_negative: 183708
vehicle_false_positive: 255
vehicle_false_negative: 0
vehicle_precision_image: 0.0
vehicle_recall_image: nan
vehicle_f1_image: 0.0
vehicle_productivity_image: 0.9986138516984394

Ge

In [4]:
print({k: list(v.keys()) for (k, v) in out_csvs.items()})

{'25079_alleyson': ['halo_productivity_combined_no_mislocalization_alleyson', 'halo_rgb_stereo_test_v6_2_alleysson', 'halo_humans_on_path_test_v6_2_8_mainline_alleysson'], '25080_alleyson': ['labelbox_import_tire_tracks_100k_diverse', 'halo_humans_on_path_test_v6_2_8_mainline', 'halo_rgb_stereo_test_v6_2', 'on_path_aft_humans_day_2024_rev2_v2', 'halo_productivity_combined']}


In [5]:
def get_metrics(df):
    fp = sum(df['human_state'] == 'false_positive')
    tn = sum(df['human_state'] == 'true_negative')
    tp_thresh = sum(df['human_state'] == 'true_positive')
    fn_thresh = sum(df['human_state'] == 'false_negative')
    try:
        tp = sum(df['n_pred_human_pixels'] >= df['min_pixels_threshold'])
        fn = sum((df['n_gt_human_pixels'] >= df['min_pixels_threshold']) & 
                  (df['n_pred_human_pixels'] < df['min_pixels_threshold']))
    except KeyError:
        tp = tp_thresh
        fn = fn_thresh

    if tp + fn < 20:
        # not enough data to give a meaningful number
        recall = float('nan')
    else:
        recall = tp / (tp + fn)
    if tp_thresh + fn_thresh < 20:
        recall_thresh = float('nan')
    else:
        recall_thresh = tp_thresh / (tp_thresh + fn_thresh)
    if fp + tn < 20:
        prod = float('nan')
    else:
        prod = tn / (fp + tn)
    return {'recall': recall, 'recall_thresh': recall_thresh, 'n': len(df), 'productivity': prod}


In [6]:
def pairwise(a):
    it = iter(a)
    last = next(it)
    for cur in it:
        yield (last, cur)
        last = cur

In [12]:
def get_metrics_plot(df):
    depth_barriers = [0, .2, .4, 1.01]
    size_barriers = [72, 200, 400, 99999]
    recall = [[0 for _ in range(len(size_barriers) - 1)] for _ in range(len(depth_barriers) - 1)]
    all_metrics = []
    for depth_ind, (depth_lo, depth_hi) in enumerate(pairwise(depth_barriers)):
        for size_ind, (size_lo, size_hi) in enumerate(pairwise(size_barriers)):
            sub_df = df[
                (df['gt_human_depth'] >= depth_lo) & 
                (df['gt_human_depth'] < depth_hi) &         
                (df['n_gt_human_pixels'] >= size_lo) &
                (df['n_gt_human_pixels'] < size_hi)]
            metrics = {**get_metrics(sub_df), 
                                'gt_human_depth': f"{depth_ind}_{int(100*depth_lo)}-{int(100*depth_hi)}",
                                'n_gt_human_pixels': f"{size_ind}_{size_lo}-{size_hi}"
                               }
            print(metrics)
            all_metrics.append(metrics)
    all_metrics = pd.DataFrame(all_metrics)
    return all_metrics


In [13]:
df_1 = out_csvs['25079_alleyson']['halo_humans_on_path_test_v6_2_8_mainline_alleysson']
df_2 = out_csvs['25080_alleyson']['halo_humans_on_path_test_v6_2_8_mainline']
df_1 = df_1[df_1['unique_id'].isin(df_2['unique_id'])].copy()
df_2 = df_2[df_2['unique_id'].isin(df_1['unique_id'])].copy()

In [14]:
def get_heatmap_ax(df):
    metrics = get_metrics_plot(df)
    metrics = metrics.pivot(index='gt_human_depth', columns='n_gt_human_pixels', values='recall')
    return seaborn.heatmap(metrics, annot=True, fmt=".3f")

In [15]:
print("ALLEYSON RECALL METRICS")
get_heatmap_ax(df_1)
get_metrics(df_1)

ALLEYSON RECALL METRICS
{'recall': nan, 'recall_thresh': nan, 'n': 8, 'productivity': nan, 'gt_human_depth': '0_0-20', 'n_gt_human_pixels': '0_72-200'}
{'recall': 0.8, 'recall_thresh': 0.8, 'n': 40, 'productivity': nan, 'gt_human_depth': '0_0-20', 'n_gt_human_pixels': '1_200-400'}
{'recall': 0.9823051171688187, 'recall_thresh': 0.9824645305276583, 'n': 6273, 'productivity': nan, 'gt_human_depth': '0_0-20', 'n_gt_human_pixels': '2_400-99999'}
{'recall': 0.6264367816091954, 'recall_thresh': nan, 'n': 174, 'productivity': 0.9806451612903225, 'gt_human_depth': '1_20-40', 'n_gt_human_pixels': '0_72-200'}
{'recall': 0.7547408343868521, 'recall_thresh': 0.8152173913043478, 'n': 791, 'productivity': 0.9752883031301482, 'gt_human_depth': '1_20-40', 'n_gt_human_pixels': '1_200-400'}
{'recall': 0.8284222305110034, 'recall_thresh': 0.9299820466786356, 'n': 2681, 'productivity': 0.98468410976388, 'gt_human_depth': '1_20-40', 'n_gt_human_pixels': '2_400-99999'}
{'recall': 0.45454545454545453, 'recal

{'recall': 0.8669235594587329,
 'recall_thresh': 0.9671379942393297,
 'n': 11334,
 'productivity': 0.9862012987012987}

In [17]:
print("NO ALLEYSON RECALL METRICS")
get_heatmap_ax(df_2)
get_metrics(df_2)

NO ALLEYSON RECALL METRICS
{'recall': nan, 'recall_thresh': nan, 'n': 7, 'productivity': nan, 'gt_human_depth': '0_0-20', 'n_gt_human_pixels': '0_72-200'}
{'recall': 0.6842105263157895, 'recall_thresh': 0.6842105263157895, 'n': 38, 'productivity': nan, 'gt_human_depth': '0_0-20', 'n_gt_human_pixels': '1_200-400'}
{'recall': 0.9802138184139142, 'recall_thresh': 0.9813307802776448, 'n': 6267, 'productivity': nan, 'gt_human_depth': '0_0-20', 'n_gt_human_pixels': '2_400-99999'}
{'recall': 0.603448275862069, 'recall_thresh': nan, 'n': 174, 'productivity': 0.9743589743589743, 'gt_human_depth': '1_20-40', 'n_gt_human_pixels': '0_72-200'}
{'recall': 0.772554002541296, 'recall_thresh': 0.8430232558139535, 'n': 787, 'productivity': 0.975609756097561, 'gt_human_depth': '1_20-40', 'n_gt_human_pixels': '1_200-400'}
{'recall': 0.8323721645520954, 'recall_thresh': 0.9343525179856115, 'n': 2601, 'productivity': 0.9838817998656817, 'gt_human_depth': '1_20-40', 'n_gt_human_pixels': '2_400-99999'}
{'reca

{'recall': 0.8646832153418765,
 'recall_thresh': 0.9672970843183609,
 'n': 11334,
 'productivity': 0.9865591397849462}

In [376]:
df_1 = out_csvs['25079_alleyson']['halo_rgb_stereo_test_v6_2_alleysson']
df_2 = out_csvs['25080_alleyson']['halo_rgb_stereo_test_v6_2']
df_1 = df_1[df_1['unique_id'].isin(df_2['unique_id'])].copy()
df_2 = df_2[df_2['unique_id'].isin(df_1['unique_id'])].copy()

In [363]:
get_metrics(df_1)

{'recall': 0.8662271373883453,
 'recall_thresh': 0.9406270847231488,
 'n': 59170,
 'productivity': 0.9773404277174025}

In [364]:
get_metrics(df_2)

{'recall': 0.8622474299893654,
 'recall_thresh': 0.9351109453663994,
 'n': 59170,
 'productivity': 0.9750433490215507}

In [377]:
df_1 = out_csvs['25079_alleyson']['halo_productivity_combined_no_mislocalization_alleyson']
df_2 = out_csvs['25080_alleyson']['halo_productivity_combined']
df_1 = df_1[df_1['unique_id'].isin(df_2['unique_id'])].copy()
df_2 = df_2[df_2['unique_id'].isin(df_1['unique_id'])].copy()

In [378]:
get_metrics(df_1)

{'recall': nan,
 'recall_thresh': nan,
 'n': 182352,
 'productivity': 0.9854512152320786}

In [379]:
get_metrics(df_2)

{'recall': nan,
 'recall_thresh': nan,
 'n': 182352,
 'productivity': 0.9896189786785996}

In [ ]:
# MAINLINE RECALL 0.9889234598630989
# MAINLINE ALLEYSON RECALL 0.9888404309902514

# STEREO TEST PRODUCITIVITY 0.7121072372870934
# STEREO TEST ALLEYSON PRODUCTIVITY 0.7014358038011453

In [ ]:
out_df = pd.read_csv(output_dir_prod + "/output.csv")

In [ ]:
prod_dset = '/data2/jupiter/datasets/halo_rgb_stereo_test_v6_0/'
master_df = pd.read_csv(prod_dset + "master_annotations.csv")

In [ ]:
fp_df = out_df[out_df['state'] == 'false_positive']
row = master_df[master_df['id'] == '64dea8faae8b0f37b46e05f7'].iloc[0]
# /mnt/sandbox1/alex.li/results/pmehta_2023_val_bestmodel/halo_rgb_stereo_test_v6_0/human_false_positive/64de93970bf522829d4cd6d8_T13_T15.png

In [ ]:
rec_artifacts = rec.get_artifacts(row)
label_converter = LabelConversion(label_map_helper)
rec_label = label_converter.convert_label_for_driveable_terrain(rec_artifacts['label'], json.loads(row['label_map']))


In [ ]:
import os
import random
dataset_path = '/mnt/sandbox1/alex.li/introspection/pmehta_2023_val_bestmodel/halo_rgb_stereo_test_v6_0/'
files = os.listdir(dataset_path + 'false_positive/')
os.mkdir(dataset_path + 'fp_subset_2/')
random.shuffle(files)
for f in files[:50]:
    os.symlink(dataset_path + 'false_positive/' + f, dataset_path + 'fp_subset_2/' + f)

In [ ]:
import os
import random
dataset_path = '/mnt/sandbox1/alex.li/introspection/pmehta_2023_val_bestmodel/halo_humans_on_path_v3/'
files = os.listdir(dataset_path + 'human_false_negative/')
os.mkdir(dataset_path + 'fn_subset/')
random.shuffle(files)
for f in files[:50]:
    os.symlink(dataset_path + 'human_false_negative/' + f, dataset_path + 'fn_subset/' + f)

In [ ]:
label_is_cutoff = []
label_is_humanonly = []
label_file = "/home/alex.li/git/JupiterCVML/europa/base/src/europa/dl/config/label_maps/eight_class_train_dust_light_as_sky_birds_as_driveable.csv"
label_map_helper = LabelMapHelper(label_file)
rec = rectifiedrgb.RectifiedRGBNPZ(prod_dset)
deb = debayeredrgb.DebayeredRGBNPZ(prod_dset)
label_converter = LabelConversion(label_map_helper)

for _, row in tqdm(master_df.iterrows(), total=len(master_df)):
    rec_artifacts = rec.get_artifacts(row)
    rec_label = label_converter.convert_label_for_driveable_terrain(rec_artifacts['label'], json.loads(row['label_map']))

    # for id in tqdm(master_df['id']):
    #     row = master_df[master_df['id'] == id].iloc[0]
    # deb_artifacts = deb.get_artifacts(row)
    # deb_label = label_converter.convert_label_for_driveable_terrain(deb_artifacts['label'], json.loads(row['label_map']))
    # rec_human = np.sum(rec_label == 5)
    # center_human = np.sum(rec_label[15:-15,15:-15] == 5)
    # edge_human = rec_human - center_human
    # deb_human = np.sum(deb_label == 5)
    # # Human is on edge only and more present in the debayered image
    # if edge_human > center_human and deb_human > rec_human * 8:
    #     #occluded human
    #     label_is_cutoff.append(True)
    # else:
    #     label_is_cutoff.append(False)
    label_is_humanonly.append(set(np.unique(rec_label)) <= set({5, 255}))

In [ ]:
sum(label_is_humanonly)

In [ ]:
master_df_full_label = master_df[~np.array(label_is_humanonly)]
master_df_full_label.to_csv(prod_dset + "master_annotations_full_label.csv")

In [ ]:
master_df_full_label = pd.read_csv(prod_dset + "master_annotations_full_label.csv")

In [ ]:
all_uid = master_df['unique_id']
other_uid = master_df_full_label['unique_id']
diff_id = set(all_uid) - set(other_uid)

In [ ]:
humanonly_df = master_df[master_df['unique_id'].isin(diff_id)]

In [ ]:
len(humanonly_df)

In [ ]:
me = pd.read_csv(prod_dset + "master_annotations_human_label.csv")
pawan = pd.read_csv("/home/alex.li/logs/only_humans_labeled_v2.csv")

In [ ]:
me['annotation_pixelwise_0_labeled_objects']

In [ ]:
pawan['id']

In [ ]:
pawan_uid = pawan['id']
me_uid = me['id']
in_pawan_only = list(set(pawan_uid) - set(me_uid))
in_me_only = list(set(me_uid) - set(pawan_uid))

print(in_pawan_only[:4])
print(in_me_only[:4])

In [ ]:
print(len(in_me_only))
print(len(in_pawan_only))